A notebook for detecting cirular tanks on GBDX using two different models. The task [circular-tank-detector](https://github.com/PlatformStories/circular-tank-detector) accepts a pansharpened image in UTM projection and executes the following:

1. Compute a max tree to select tank candidates 
2. Chip tank candidates from the input image
3. Deploy the input or default CNN to classify candidates as 'tank' or 'other'

First we use a model that was trained on data selected by the max-tree and then manually curated, followed by execution by a model trained on data selected only using the max-tree.

In [ ]:
from gbdxtools import Interface
from os.path import join
import uuid

In [ ]:
gbdx = Interface()
input_location = 's3://gbd-customer-data/32cbab7a-4307-40c8-bb31-e2de32f940c2/platform-stories/circular-tank-detector/'

# Generate output location
random_str = str(uuid.uuid4())
output_location = join('platform-stories/trial-runs', random_str)

In [ ]:
# Create a task to deploy the manual (default) model
ctd_manual = gbdx.Task('circular-tank-detector')
ctd_manual.inputs.ps_image = join(input_location, 'ps-image')
ctd_manual.inputs.min_size = '50'  # minimum acceptable area for tank candidates

In [ ]:
wf_manual = gbdx.Workflow([ctd_manual])

# Save output geojsons
wf_manual.savedata(ctd_manual.outputs.detections, join(output_location, 'tank_detections_manual'))
wf_manual.savedata(ctd_manual.outputs.candidates, join(output_location, 'tank_candidates_manual'))

# Execute the workflow
wf_manual.execute()

In [ ]:
# Create a task that uses the unsupervised model
ctd_unsupervised = gbdx.Task('circular-tank-detector')
ctd_unsupervised.inputs.ps_image = join(input_location, 'ps-image')
ctd_unsupervised.inputs.model = join(input_location, 'models', 'unsupervised') # location of unsupervised model to use
ctd_unsupervised.inputs.min_size = '50'  # minimum acceptable area for tank candidates

In [ ]:
wf_unsupervised = gbdx.Workflow([ctd_unsupervised])

# Save output geojsons
wf_unsupervised.savedata(ctd_unsupervised.outputs.detections, join(output_location, 'tank_detections_unsupervised'))
wf_unsupervised.savedata(ctd_unsupervised.outputs.candidates, join(output_location, 'tank_candidates_unsupervised'))

# Execute the workflow
wf_unsupervised.execute()

In [ ]:
# Check status of first workflow
wf_manual.status

In [ ]:
# Check status of second workflow
wf_unsupervised.status